In [1]:
import pandas as pd
import numpy as np
import config
import data_process.neg_sample as ng_sample
from sklearn import metrics, preprocessing
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import tqdm
import torch.nn.functional as F
import random

In [17]:
train_data = pd.read_csv(
    config.train_rating,
    sep='\t', header=None, names=['user', 'item'],
    usecols=[0, 1], dtype={0: np.int32, 1: np.int32})

In [20]:
train_data['item'].max()

3705

In [31]:
aa = set(train_data['item'])

In [ ]:
test_data_pos = pd.read_csv(
    config.test_rating,
    sep='\t', header=None, names=['user', 'item'],
    usecols=[0, 1], dtype={0: np.int32, 1: np.int32})

# user_num = train_data['user'].max() + 1
user_num = len(set(train_data['user']))
item_num = len(set(train_data['item']))
train_data = train_data.values.tolist()
test_data_pos = test_data_pos.values.tolist()
    
    
    

In [24]:
test_data = []
with open(config.test_negative, 'r') as fd:
    line = fd.readline()
    while line != None and line != '':
        arr = line.split('\t')
        u = eval(arr[0])[0]
        test_data.append([u, eval(arr[0])[1]])
        for i in arr[1:]:
            test_data.append([u, int(i)])
        line = fd.readline()

In [32]:
bb= set([i[1] for i in test_data])

In [38]:
cc =aa.union(bb)

In [39]:
len(cc)

3706

In [5]:
user_num

6040

In [8]:
train_data[1:5]

[[0, 34], [0, 4], [0, 35], [0, 30]]

In [11]:
max([i[1] for i in train_data])

3705

In [14]:
min([i[1] for i in train_data])

0

In [13]:
len(set(i[1] for i in train_data))

3704

In [164]:
all_data = pd.read_csv(
    "/Users/hao/Documents/MA_thesis/ncf-torch/Data/mini_split.csv",
    header=0, names=['user', 'item', 'rating', 'split'])

In [165]:
all_data

,user,item,rating,split
0,1,1,1,Train
1,1,2,1,Test
2,1,5,1,Test
3,1,4,1,Train
4,2,1,1,Test
5,2,2,1,Train
6,3,2,1,Train
7,3,5,1,Train
8,3,6,1,Test
9,4,1,1,Test


In [166]:
train_data = all_data[all_data['split']=='Train']

In [167]:
test_data = all_data[all_data['split']=='Test']

## Creat the random sample function

In [168]:
all_item = set(all_data.item)

In [181]:
train_data

,user,item,rating,split
0,1,1,1,Train
3,1,4,1,Train
5,2,2,1,Train
6,3,2,1,Train
7,3,5,1,Train
10,4,3,1,Train
11,4,4,1,Train


In [182]:
test_data

,user,item,rating,split
1,1,2,1,Test
2,1,5,1,Test
4,2,1,1,Test
8,3,6,1,Test
9,4,1,1,Test
12,5,3,1,Test


In [171]:
test_gt = test_data[['user', 'item']]

In [172]:
import random
import tqdm

In [173]:
item_diff =all_item - {2}

In [174]:
random.sample(item_diff, 2)

[5, 6]

In [189]:
def _neg_sample(test_gt, all_data, all_item, ng_num):
    neg_set = []
    for i in tqdm.tqdm(test_gt.values):
        train_user_item = set(all_data[all_data['user']==i[0]]['item'])
        item_diff = all_item - train_user_item
        random.seed(10)
        neg = random.sample(item_diff, ng_num)
        for j in neg:
            neg_set.append([i[0],j])
    df_neg = pd.DataFrame(neg_set, columns=['user', 'item'])
    df_neg.to_csv('../Data/mini_split_neg.csv')
#     return neg_set


In [190]:
_neg_sample(test_gt, all_data, all_item, ng_num=2)

100%|██████████| 6/6 [00:00<00:00, 1051.16it/s]


In [89]:
df_jrs = pd.read_feather('/Users/hao/Documents/MA_thesis/ncf-torch/Data/jobs/apps_sub', columns={'UserID', 'JobID', 'Split'})

In [90]:
df_jrs.head(3)

,JobID,Split,UserID
0,309823,Train,7
1,703889,Train,7
2,473209,Train,106


In [211]:
df_jrs.rename(columns = {'UserID':'user', 'JobID':'item', 'Split':'split'}, inplace=True )

In [212]:
df_jrs

,item,split,user
0,309823,Train,7
1,703889,Train,7
2,473209,Train,106
3,1052756,Train,108
4,960067,Train,132
...,...,...,...
683889,932512,Train,1472005
683890,532537,Train,1472008
683891,12087,Train,1472008
683892,473827,Train,1472035


In [204]:
df_jrs['JobID'] = df_jrs['JobID'].astype('int64')

In [205]:
df_jrs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683894 entries, 0 to 683893
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   JobID   683894 non-null  int64 
 1   Split   683894 non-null  string
 2   UserID  683894 non-null  Int64 
dtypes: Int64(1), int64(1), string(1)
memory usage: 16.3 MB


# subsample as dataset 

In [78]:
all_data = pd.read_feather("/Users/hao/Documents/MA_thesis/ncf-torch/Data/jobs/apps_pos_sub")

In [79]:
all_data.head(2)

,UserID,WindowID,Split,ApplicationDate,JobID,Applies
0,7,3,Train,2012-04-22 08:14:26.443,309823,1
1,7,3,Train,2012-04-28 12:17:58.630,703889,1


In [33]:
# list(all_data.UserID)

In [80]:
all_data['JobID'].unique()

<StringArray>
[ '309823',  '703889',  '473209', '1052756',  '960067',  '828884',  '569294',
  '888911',  '585734',  '577855',
 ...
  '641881',  '127237',  '123658',   '85522',  '480993', '1059261', '1056469',
  '932512',  '532537',   '12087']
Length: 152923, dtype: string

In [26]:
selected = [7, 106, 212]

In [27]:
sub_data = all_data[all_data['UserID'].isin(selected)]

,UserID,WindowID,Split,ApplicationDate,JobID,Applies
0,7,3,Train,2012-04-22 08:14:26.443,309823,1
1,7,3,Train,2012-04-28 12:17:58.630,703889,1
2,106,3,Train,2012-04-27 11:41:22.320,473209,1
7,212,3,Train,2012-04-04 16:06:28.207,888911,1
8,212,3,Train,2012-05-01 02:03:50.473,585734,1
9,212,3,Train,2012-05-03 19:48:55.650,577855,1
10,212,3,Train,2012-04-30 13:06:57.887,224895,1


In [29]:
pd.factorize(all_data['Split'])

(array([0, 0, 0, ..., 0, 0, 0]), Index(['Train', 'Test'], dtype='object'))

In [36]:
all_data[all_data['UserID']==1471408]

,UserID,WindowID,Split,ApplicationDate,JobID,Applies
683805,1471408,5,Test,2012-05-30 21:25:20.240,900240,1
683806,1471408,5,Test,2012-05-30 21:26:18.227,611427,1
683807,1471408,5,Test,2012-05-30 21:34:18.823,996746,1
683808,1471408,5,Test,2012-05-30 21:40:40.707,217832,1
683809,1471408,5,Test,2012-05-30 21:41:01.220,801645,1
683810,1471408,5,Test,2012-05-12 20:41:42.947,296442,1
683811,1471408,5,Test,2012-05-12 20:42:17.780,200315,1


In [30]:
all_data[all_data['Split'] == 'Test']

,UserID,WindowID,Split,ApplicationDate,JobID,Applies
48,883,3,Test,2012-04-27 21:27:27.733,663823,1
133,1694,3,Test,2012-04-16 12:45:40.567,653716,1
134,1694,3,Test,2012-04-16 12:55:12.100,881830,1
135,1694,3,Test,2012-04-04 16:14:55.160,464459,1
136,1694,3,Test,2012-04-16 12:53:34.940,131773,1
...,...,...,...,...,...,...
683807,1471408,5,Test,2012-05-30 21:34:18.823,996746,1
683808,1471408,5,Test,2012-05-30 21:40:40.707,217832,1
683809,1471408,5,Test,2012-05-30 21:41:01.220,801645,1
683810,1471408,5,Test,2012-05-12 20:41:42.947,296442,1


In [37]:
# create leave one out

In [38]:
all_data.head(5)

,UserID,WindowID,Split,ApplicationDate,JobID,Applies
0,7,3,Train,2012-04-22 08:14:26.443,309823,1
1,7,3,Train,2012-04-28 12:17:58.630,703889,1
2,106,3,Train,2012-04-27 11:41:22.320,473209,1
3,108,3,Train,2012-05-01 22:47:18.830,1052756,1
4,132,3,Train,2012-04-21 01:25:58.853,960067,1


In [39]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 683894 entries, 0 to 683893
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   UserID           683894 non-null  Int64         
 1   WindowID         683894 non-null  Int64         
 2   Split            683894 non-null  string        
 3   ApplicationDate  683894 non-null  datetime64[ns]
 4   JobID            683894 non-null  string        
 5   Applies          683894 non-null  int64         
dtypes: Int64(2), datetime64[ns](1), int64(1), string(2)
memory usage: 32.6 MB


In [44]:
sort_all_data = all_data.sort_values(['UserID','ApplicationDate'])

In [51]:
sort_all_data

,UserID,WindowID,Split,ApplicationDate,JobID,Applies
0,7,3,Train,2012-04-22 08:14:26.443,309823,1
1,7,3,Train,2012-04-28 12:17:58.630,703889,1
217568,9,4,Train,2012-05-14 22:30:42.090,809208,1
217569,9,4,Train,2012-05-14 22:38:37.243,136489,1
217570,9,4,Train,2012-05-14 22:48:09.997,617374,1
...,...,...,...,...,...,...
217566,1472079,3,Train,2012-05-03 12:59:11.900,197942,1
217567,1472079,3,Train,2012-05-12 21:23:48.107,259504,1
683893,1472085,5,Train,2012-05-28 10:38:15.427,849565,1
451459,1472090,4,Train,2012-05-12 16:40:50.483,209535,1


In [46]:
leave_one_out_test = sort_all_data.groupby('UserID').last().reset_index()

In [48]:
len(set(leave_one_out_test['UserID']))

134102

In [55]:
leave_one_out_train = sort_all_data[~sort_all_data['ApplicationDate'].isin(leave_one_out_test['ApplicationDate'])]

In [56]:
len(set(leave_one_out_train['UserID']))

90169

Filter out any users or items in the test set that do not appear in the training set.

In [61]:
test = leave_one_out_test[leave_one_out_test["UserID"].isin(leave_one_out_train["UserID"].unique())]
test = test[test["JobID"].isin(leave_one_out_train["JobID"].unique())]

In [63]:
len(set(test['UserID']))

74291

In [71]:
test = pd.read_feather("../Data/jobs/leave_one_test")

In [74]:
test

,index,UserID,WindowID,Split,ApplicationDate,JobID,Applies
0,0,7,3,Train,2012-04-28 12:17:58.630,703889,1
1,1,9,4,Train,2012-05-14 22:48:09.997,617374,1
2,2,14,4,Train,2012-05-15 14:35:51.237,663552,1
3,3,16,5,Train,2012-06-21 11:35:54.980,747203,1
4,4,24,5,Train,2012-05-29 03:12:33.070,335132,1
...,...,...,...,...,...,...,...
74286,134082,1471887,5,Train,2012-05-27 14:07:09.343,751714,1
74287,134091,1471992,4,Train,2012-05-14 12:23:28.793,300837,1
74288,134093,1471998,5,Train,2012-06-05 22:03:21.123,434075,1
74289,134097,1472036,4,Train,2012-05-14 15:56:13.190,443782,1


In [75]:
train = pd.read_feather("../Data/jobs/leave_one_train")

In [76]:
train

,index,UserID,WindowID,Split,ApplicationDate,JobID,Applies
0,0,7,3,Train,2012-04-22 08:14:26.443,309823,1
1,217568,9,4,Train,2012-05-14 22:30:42.090,809208,1
2,217569,9,4,Train,2012-05-14 22:38:37.243,136489,1
3,217571,14,4,Train,2012-04-08 18:49:15.563,574999,1
4,217572,14,4,Train,2012-05-10 14:03:09.333,372423,1
...,...,...,...,...,...,...,...
549516,217563,1472079,3,Train,2012-04-17 12:36:24.737,54908,1
549517,217564,1472079,3,Train,2012-05-03 12:51:01.167,424017,1
549518,217565,1472079,3,Train,2012-05-03 12:52:23.950,10703,1
549519,217566,1472079,3,Train,2012-05-03 12:59:11.900,197942,1


In [82]:
len(set(train['JobID']))

130454

In [85]:
x = torch.randn(2, 3)

In [86]:
torch.cat((x,x,x), -1)

tensor([[ 2.0313,  0.3934, -0.6048,  2.0313,  0.3934, -0.6048,  2.0313,  0.3934,
         -0.6048],
        [ 0.7225, -0.9519,  0.6945,  0.7225, -0.9519,  0.6945,  0.7225, -0.9519,
          0.6945]])

In [88]:
torch.cat((x,x,x), 1)

tensor([[ 2.0313,  0.3934, -0.6048,  2.0313,  0.3934, -0.6048,  2.0313,  0.3934,
         -0.6048],
        [ 0.7225, -0.9519,  0.6945,  0.7225, -0.9519,  0.6945,  0.7225, -0.9519,
          0.6945]])

In [4]:
data_ori  = ng_sample.read_feather("../Data/jobs/leave_one_train")
test_ori = ng_sample.read_feather("../Data/jobs/leave_one_test")
test_neg = pd.read_csv("../Data/jobs/apps_neg.csv")

In [8]:
data_ori['rating'] = 1
# test_neg['rating'] = 0
data_ori['flag'] = 1
test_neg['flag'] = -1

Create negtive samples for training

In [9]:
test_neg.shape

(7503391, 4)

In [10]:
df_all = pd.concat([data_ori, test_neg], axis=0)

In [11]:
le = preprocessing.LabelEncoder()

In [12]:
features = data_ori.columns
features = [i for i in features if i not in ['flag', 'rating']]
features

['user', 'item']

In [13]:
for f in features:
    df_all[f] = le.fit_transform(df_all[f])

In [14]:
df_train = df_all[df_all.flag==1]
df_test = df_all[df_all.flag==-1]

In [15]:
df_train=df_train.drop(['flag'], axis=1)
df_test=df_test.drop(['flag'], axis=1)

In [16]:
# df_test.loc[df_test.groupby('user')['rating'].head(1).index, 'rating'] = 1
# df_test.head()

Create negative samples for training

In [17]:
df_test = pd.DataFrame([[1,2], [1,4], [2,5]], columns=['user', 'item'])
df_test

,user,item
0,1,2
1,1,4
2,2,5


In [135]:
ng_num = 2

In [204]:
all_user = set([i for i in range(100)])

In [205]:
random.sample(all_user, ng_num)

[11, 98]

In [206]:
def sample_ng_train(x, all_user, ng_num):
    group_items = set(x.item)
    sub = all_user - group_items
    random.seed(123)
    ng_samples = random.sample(sub, ng_num*len(group_items))
    return ng_samples

In [209]:
series_ng_sample = df_test.groupby('user').apply(lambda x: test_func(x, all_user, ng_num))

In [215]:
aa =pd.DataFrame(series_ng_sample, columns=['item']).reset_index(level=0)

In [224]:
flat_list = []
for i in bb:
#     print(i[0])
#     print(i[1])
    for j in i[1]:
        flat_list.append([i[0], j, 0])

In [226]:
cc =pd.DataFrame(flat_list, columns=['user', 'item', 'rating'])
cc

,user,item,rating
0,1,8,0
1,1,36,0
2,1,13,0
3,1,54,0
4,2,7,0
5,2,35,0


In [228]:
df_test['rating'] = 1

In [232]:
df_pos_neg = pd.concat([df_test, cc], axis = 0)

In [158]:
# df_test.groupby('user').apply(rand_sample, args=[ng_num], axis=1)

In [236]:
df_pos_neg.sort_values(['user'])

,user,item,rating
0,1,2,1
1,1,4,1
0,1,8,0
1,1,36,0
2,1,13,0
3,1,54,0
2,2,5,1
4,2,7,0
5,2,35,0


In [19]:
df_train.shape


(549521, 3)

In [2]:
from data_process.neg_sample import random_sample

In [14]:
random_sample(test_gt=df_test, all_data=df_all, ng_num=4, path_save='test1.csv', train=True)

100%|██████████| 3/3 [00:00<00:00, 51.94it/s]


In [44]:
class Entity(nn.Module):
    def __init__(self, embedding_size, num_numerical_cols, 
                 output_size, layers=[5], p = 0.4):
        '''
        embedding_size: Contains the embedding size for the categorical columns
        num_numerical_cols: Stores the total number of numerical columns
        output_size: The size of the output layer or the number of possible outputs.
        layers: List which contains number of neurons for all the layers.
        p: Dropout with the default value of 0.5
        
        '''
        super(entity, self).__init__()
        #list of ModuleList objects for all categorical columns
        self.all_embeddings = nn.ModuleList([nn.Embedding(ni, nf) for ni, nf in embedding_size])
        
        #drop out value for all layers
        self.embedding_dropout = nn.Dropout(p)
        
        #list of 1 dimension batch normalization objects for all numerical columns
#         self.batch_norm_num = nn.BatchNorm1d(num_numerical_cols)

        #the number of categorical and numerical columns are added together and stored in input_size
        all_layers = nn.ModuleList() 
        num_categorical_cols = sum((nf for ni, nf in embedding_size))
        input_size = num_categorical_cols + num_numerical_cols
        
        #loop iterates to add corresonding layers to all_layers list above
        for i in layers:
            all_layers.append(nn.Linear(input_size, i))
            all_layers.append(nn.ReLU(inplace=True))
#             all_layers.append(nn.BatchNorm1d(i))
            all_layers.append(nn.Dropout(p))
            input_size = i
            
        #append output layer to list of layers    
        all_layers.append(nn.Linear(layers[-1], output_size))
        
        #pass all layers to the sequential class
        self.layers = nn.Sequential(*all_layers)
        
#     def _init_weight_(self):

    def forward(self, x_categorical, x_numerical):
        #this starts the embedding of categorical columns
        embeddings = []
        for i,e in enumerate(self.all_embeddings):
            embeddings.append(e(x_categorical[:,i]))

        x = torch.cat(embeddings, 1)
#         print(x.shape)
        x = self.embedding_dropout(x)

        #normalizing numerical columns
#         x_numerical = self.batch_norm_num(x_numerical)
        #concatenating numerical and categorical columns        
#         x_numerical = x_numerical.reshape(-1,1)
#         print('hao--------1', x.shape)
#         print('hao--------2',x_numerical.shape)
        x = torch.cat([x, x_numerical], 1)
        x = self.layers(x)
        x = torch.sigmoid(x)
        return x

In [ ]:
class catData(data.Dataset):
    '''
    num_f: the numerical features
    cat_f: the categorical features 
    '''
    def __init__(self, inp_data, label_name, num_f=None, cat_f=None):
        self.data = inp_data
        self.label = inp_data[label_name]
        self.cat_f = cat_f
        self.num_f = num_f
    def __len__(self):
        return len(self.label)
    def __getitem__(self, idx):
#         num_data = np.array(self.data.iloc[idx, 0]).reshape(-1,1)
#         cat_data = np.array(self.data.iloc[idx, 1:-1])
        if self.cat_f not None:
            num_data = np.array(self.data[self.num_f])[idx]
        if self.num_f not None:
            cat_data = np.array(self.data[self.cat_f])[idx]
        label = self.label[idx]
#         print('hao-------!!', label)
        return num_data, cat_data, label
        

In [ ]:
label_name = 'rating'

In [1]:
import pandas as pd
import numpy as np
# import config
import data_process.neg_sample as ng_sample
from sklearn import metrics, preprocessing
import torch
import torch.nn as nn
import torch.utils.data as data
import torch.optim as optim
import tqdm
import torch.nn.functional as F
import random
import evaluate

In [2]:
df_train1  = ng_sample.read_feather("../Data/jobs/leave_one_train")
df_train2 = pd.read_feather("../Data/jobs/pos_neg_train_uncode0")
df_test = pd.read_csv("../Data/jobs/apps_neg.csv")

In [3]:
df_train1['rating'] = 1
df_train_all = pd.concat([df_train1, df_train2], axis=0)
df_train_all['flag'] = 1
df_test['flag'] = -1
df_all = pd.concat([df_train_all, df_test], axis=0)

In [4]:
data_input_path = '/home/hao/Documents/MA_thesis_win/hao_jrs/data/clean/sub'

In [5]:
apps = pd.read_feather(data_input_path + '/users_sub')

In [6]:
apps.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161351 entries, 0 to 161350
Data columns (total 15 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   UserID                161351 non-null  Int64         
 1   WindowID              161351 non-null  Int64         
 2   Split                 161351 non-null  string        
 3   City                  161351 non-null  string        
 4   State                 161135 non-null  string        
 5   Country               161351 non-null  string        
 6   ZipCode               160636 non-null  string        
 7   DegreeType            161351 non-null  string        
 8   Major                 121184 non-null  string        
 9   GraduationDate        111725 non-null  datetime64[ns]
 10  WorkHistoryCount      161351 non-null  Int64         
 11  TotalYearsExperience  155670 non-null  Int64         
 12  CurrentlyEmployed     142860 non-null  string        
 13 

In [7]:
apps.rename(columns = {'UserID':'user'}, inplace=True)

In [8]:
df_all.shape

(10250996, 4)

In [10]:
aa = pd.merge(df_all, apps, how='left', on=['user'])

In [11]:
aa.shape

(10250996, 18)

In [12]:
aa.to_feather("test1")

In [13]:
aa.columns

Index(['user', 'item', 'rating', 'flag', 'WindowID', 'Split', 'City', 'State',
       'Country', 'ZipCode', 'DegreeType', 'Major', 'GraduationDate',
       'WorkHistoryCount', 'TotalYearsExperience', 'CurrentlyEmployed',
       'ManagedOthers', 'ManagedHowMany'],
      dtype='object')

In [14]:
aa.head()

,user,item,rating,flag,WindowID,Split,City,State,Country,ZipCode,DegreeType,Major,GraduationDate,WorkHistoryCount,TotalYearsExperience,CurrentlyEmployed,ManagedOthers,ManagedHowMany
0,7,309823,1,1,3,Train,Roanoke,VA,US,24012,High School,Not Applicable,1995-01-01,2,13,Yes,No,0
1,9,809208,1,1,4,Train,Houston,TX,US,77095,High School,Not Applicable,NaT,3,3,Yes,No,0
2,9,136489,1,1,4,Train,Houston,TX,US,77095,High School,Not Applicable,NaT,3,3,Yes,No,0
3,14,574999,1,1,4,Train,San Antonio,TX,US,78247,None,General,NaT,2,10,No,No,0
4,14,372423,1,1,4,Train,San Antonio,TX,US,78247,None,General,NaT,2,10,No,No,0
